# Reed-Solomon codes

<u> Naam: Ferre Van der Vieren (r0851455)</u> 

In deze notebook zullen we trachten de Reed-Solomon code te implementeren. 

## Coderen

Implementeer de method $\texttt{encode(w, a, p)}$ die een informatiewoord $w$ van elementen over het veld $\mathbb{Z}_p$ omzet in een codewoord, gegeven de lijst van interpolatiepunten $a$.

In [192]:
import numpy as np

def encode(w, a, p):
    F = np.vander(a, len(w), increasing = True)
    c = F.dot(w) % p
    return c

w = np.array([1,2,3])
a = np.array([1,2,3,4,5,6,7])
p = 11
print((encode(w,a,p)==np.array([6, 6, 1, 2, 9, 0, 8])).all())

True


## Decoderen

Implementeer hieronder een methode om een lijst van codewoorden te ontcijferen. Je hebt opnieuw Gauss eliminatie over het veld $\mathbb{Z}_p$ nodig om de veeltermen $G(x)$ en $E(x)$ te vinden, dit kan je kopiëren uit vorige notebook, dezelfde opmerking geldt.

In [193]:
# @return
    #  if (bezout == False): gcd(a, b)
    #  if (bezout == True): Bezout's identity coefficients s and t for which gcd(a, b) = s*a + t*b
def eaa(x, y, bezout = True):
    (a, new_a) = (x, y)
    (b, new_b) = (1, 0)
    (c, new_c) = (0, 1)
    
    while new_a != 0:
        q = a // new_a  # devide then .floor
        
        # coefficients
        (a, new_a) = (new_a, a - q * new_a)
        (b, new_b) = (new_b, b - q * new_b)
        (c, new_c) = (new_c, c - q * new_c)
        
    if (bezout):
        return (b, c)
    return a

def gcd(a, m):
    return eaa(a, m, False)

def inv(a, q):
    if (gcd(a, q) != 1):  # if gcd(a, q) != 1, then a is not a unit
        return False
    b = eaa(a, q)
    return (b[0] % q)
    
def gauss(A, p):
    dim = A.shape
    
    if (dim[0] > dim[1]): # F must be transposed
        A = np.transpose(A)
        dim = A.shape

    for i in range(dim[0]):
        if A[i][i] != 0:
            A[i] = np.dot(A[i], inv(A[i][i], p))

            # 
            for j in range(dim[0]):
                if i == j:  # skip pivot row
                    continue
                A[j] = np.add(A[j], np.dot(A[i], -1 * A[j][i])) % p

    return A % p

Nadat je $G(x)$ en $E(x)$ gevonden hebt, kan je $W(x)$ vinden door $G(x)$ te delen door $E(x)$. Hiervoor kan je de staartdeling voor veeltermen gebruiken. Implementeer de methode $\texttt{long_div(a, b, p)}$ die de veelterm $a(x)$ deelt door $b(x)$ over het veld $\mathbb{Z}_p$. *Hint: de klasse numpy.polynomial.Polynomial heeft een aantal ingebouwde veeltermbewerkingen en vermindert het implementatiewerk.*

In [194]:
from numpy import polynomial

def long_div(a, b, p):
    q, r = np.polydiv(a, b)
    return np.mod(q, p)

Implementeer vervolgens de methode $\texttt{decode(c, a, n, p)}$ die het codewoord $c$ waarbij al dan niet fouten zijn ontstaan bij de transmissie decodeert naar zijn originele vorm.

In [195]:
def decode(c, a, n, p):
    N = len(a)
    e = (N - n)//2  # since N = n + 2e
    M = np.zeros([N, N+1]).astype(int)
    
    for i in range(N):
        # first n+e kolommen of M (only dependent on interpolation points)
        for j in range(n + e):
            M[i][j] = a[i]**j
        # remaining columns of M (dependend on received c)
        for j in range(n + e, N + 1):
            M[i][j] = (-c[i]) * (a[i]**(j - (n+e)))
    
    A = gauss(M, p)
    k = np.sum(~A.any(1))   # k := amount of zero rows
    
    # calculate x and y by extracting columns after Gauss elimination
    x = -(A[0: N - k, N - k])    
    y = [1] + k * [0]
    
    coefficients = np.concatenate((x,y))  # concatenate x and y to get coefficients
    
    # coefficients of G are first n+e elements
    t = np.flip(coefficients[0:n+e], 0)
    G = np.poly1d(t)
    
    s = np.flip(coefficients[n + e: N + 1], 0)
    E = np.poly1d(s)
    
    W = long_div(G, E, p)
    return np.array(W[::-1])   # flip array

Als alles goed gegaan is zou hier nu enkel $\texttt{True}$ mogen verschijnen.

In [196]:
w = [1,3,5,6]
p = 11
N = 8
a = [1,2,3,4,5,6,7,8]
c = encode(w, a, p)

# Decoderen zonder fouten:
print((w == decode(c, a, 4, p)).all())

# Decoderen met het maximaal aantal fouten:
c[0] += 3
c[2] += 9
c %= p

print((w == decode(c, a, 4, p)).all())

True
True
